In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false; // disable scroll bar when displaying Folium map
}

<IPython.core.display.Javascript object>

# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Global Historical Climatology Network daily (GHCNd)](https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-daily) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe - it's a wonderfully large dataset to play with! In particular, you will be asked to use data from the Ann Arbor Michigan location (my home!). and this is stored in the file: `assets/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`

Each row in this datafile corresponds to a single observation from a weather station, and has the following variables:
* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write a python notebook which plots line graphs of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015. (Based on the graph, do you think extreme weather is getting more frequent in 2015?)
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

I've written some steps I think would be good to go through, but there are other ways to solve this assignment so feel free to explore the pandas library! What I really want to see is an image that looks like this sketch I drew at my desk:

![](assets/chris_sketch.png)

In [2]:
#  I'll be using the folium package to render the data into a map in Jupyter.

import folium
import pandas as pd

# get the location information for this dataset
df = pd.read_csv('assets/BinSize_d400.csv')
station_locations_by_hash = df[df['hash'] == 'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89']

# get longitude and lattitude to plot
lons = station_locations_by_hash['LONGITUDE'].tolist()
lats = station_locations_by_hash['LATITUDE'].tolist()

# plot on a beautiful folium map
my_map = folium.Map(location = [lats[0], lons[0]], height = 500,  zoom_start = 9)
for lat, lon in zip(lats, lons):
    folium.Marker([lat, lon]).add_to(my_map)

# render map in Jupyter
display(my_map)

## Step 1
Load the dataset and transform the data into Celsius (refer to documentation) then extract all of the rows which have minimum or maximum temperatures.

__hint: when I did this step I had two DataFrame objects, each with ~80,000 entries in it__

In [2]:
import pandas as pd
df = pd.read_csv('assets/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df.head()

,ID,Date,Element,Data_Value
0,USW00094889,2014-11-12,TMAX,22
1,USC00208972,2009-04-29,TMIN,56
2,USC00200032,2008-05-26,TMAX,278
3,USC00205563,2005-11-11,TMAX,139
4,USC00200230,2014-02-27,TMAX,-106


In [43]:
# In this code cell, transform the Data_Value column
df["Celsius"] = (df["Data_Value"] - 32)*5/9
max_temps = df[df["Element"] == "TMAX"]
min_temps = df[df["Element"] == "TMIN"]
max_temps.shape, min_temps.shape

((83063, 5), (82022, 5))

## Step 2
In order to visualize the data we would plot the min and max data for each day of the year between the years 2005 and 2014 across all weather stations. But we also need to find out when the min or max temperature in 2015 falls below the min or rises above the max for the previous decade.

If you did step 1 you have two Series objects with min and max times for the years 2005 through 2015. You can use Pandas `groupby` to create max and min temperature Series objects across all weather stations for each day of these years, and you can deal with the records for February 29 (the leap year) by dropping them.

__hint: when I finished this step, I had two DataFrame objects, each with exactly 4015 observations in them__

In [44]:
# create a DataFrame of maximum temperature by date

max_temps["Date"] = pd.to_datetime(max_temps["Date"])
min_temps["Date"] = pd.to_datetime(min_temps["Date"])
max_temps["Month"] = max_temps["Date"].dt.month
max_temps["Day"] = max_temps["Date"].dt.day
min_temps["Month"] = min_temps["Date"].dt.month
min_temps["Day"] = min_temps["Date"].dt.day
max_temps = max_temps[~((max_temps["Month"] == 2) & (max_temps["Day"] == 29))]
min_temps = min_temps[~((min_temps["Month"] == 2) & (min_temps["Day"] == 29))]

max_temps_by_date = max_temps.groupby("Date").max()
# create a DataFrame of minimum temperatures by date
min_temps_by_date = min_temps.groupby("Date").min()

max_temps_by_date.shape, min_temps_by_date.shape

/var/folders/hc/dvc94mw13w5gvc0rlv0rbzbw0000gn/T/ipykernel_92846/1213919489.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_temps["Date"] = pd.to_datetime(max_temps["Date"])
/var/folders/hc/dvc94mw13w5gvc0rlv0rbzbw0000gn/T/ipykernel_92846/1213919489.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_temps["Date"] = pd.to_datetime(min_temps["Date"])
/var/folders/hc/dvc94mw13w5gvc0rlv0rbzbw0000gn/T/ipykernel_92846/1213919489.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

((4015, 6), (4015, 6))

## Step 3
Now that you have grouped the daily max and min temperatures for each day of the years 2005 through 2015, you can separate out the data for 2015. Then you can use the Pandas `groupby` function to find the max and min of the temperature data for each __day of the year__ for the 2005-2014 data.

__hint: at the end of this step I had two DataFrames, one of maximum and the other of minimum values, which each had 365 observations in them. I also had another pair of similar DataFrames but only for the year 2015.__

In [46]:
# calculate the minimum and maximum values for the day of the year for 2005 through 2014
max_temps_2005_2014 = max_temps[(max_temps["Date"] >= "2005-01-01") & (max_temps["Date"] < "2014-12-31")]
min_temps_2005_2014 = min_temps[(min_temps["Date"] >= "2005-01") & (min_temps["Date"] <= "2014-12-31")]
max_temps_2005_2014 = max_temps_2005_2014.groupby(["Month", "Day"]).max()
min_temps_2005_2014 = min_temps_2005_2014.groupby(["Month", "Day"]).min()

# calculate the minimum and maximum values for the years 2015
max_temps["Year"] = max_temps["Date"].dt.year
min_temps["Year"] = min_temps["Date"].dt.year
max_temp_2015 = max_temps[max_temps["Year"] == 2015]
max_temps_2015 = max_temp_2015.groupby(["Month", "Day"]).max()
min_temps_2015 = min_temps[min_temps["Year"] == 2015]
min_temps_2015 = min_temps_2015.groupby(["Month", "Day"]).min()
max_temps_2015.shape, min_temps_2015.shape

((365, 6), (365, 6))

## Step 4
Now it's time to plot! You need to explore matplotlib in order to plot line graphs of the min and max temperatures for the years 2005 through 2014 and to scatter plot __only__ the daily 2015 temperatures that exceeded those values.

In [ ]:
import matplotlib.pyplot as plt
from calendar import month_abbr

# put your plotting code here!